In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('../raw_data/text_emotion.csv')

In [3]:
data.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [5]:
data['sentiment'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [24]:
test_data = pd.read_csv('../raw_data/data_test.csv')
train_data = pd.read_csv('../raw_data/data_train.csv')

In [10]:
train_data.head()

,Emotion,Text
0,neutral,There are tons of other paintings that I thin...
1,sadness,"Yet the dog had grown old and less capable , a..."
2,fear,When I get into the tube or the train without ...
3,fear,This last may be a source of considerable disq...
4,anger,She disliked the intimacy he showed towards so...


In [13]:
train_data['Emotion'].value_counts()

sadness    1641
joy        1619
neutral    1616
anger      1566
fear       1492
Name: Emotion, dtype: int64

In [15]:
train_data.loc[0]

Emotion                                              neutral
Text        There are tons of other paintings that I thin...
Name: 0, dtype: object

In [26]:
train_data['clean_text'] = train_data['Text'].apply(lambda x: x.lower())

In [30]:
train_data['clean_text'] = train_data['clean_text'].apply(
    lambda x: ''.join(let for let in x if not let.isdigit())
)

0       <generator object <genexpr> at 0x12600a5d0>
1       <generator object <genexpr> at 0x12600a5d0>
2       <generator object <genexpr> at 0x12600a5d0>
3       <generator object <genexpr> at 0x12600a5d0>
4       <generator object <genexpr> at 0x12600a5d0>
                           ...                     
7929    <generator object <genexpr> at 0x12600a5d0>
7930    <generator object <genexpr> at 0x12600a5d0>
7931    <generator object <genexpr> at 0x12600a5d0>
7932    <generator object <genexpr> at 0x12600a5d0>
7933    <generator object <genexpr> at 0x12600a5d0>
Name: Text, Length: 7934, dtype: object

In [31]:
train_data

,Emotion,Text,clean_text
0,neutral,There are tons of other paintings that I thin...,there are tons of other paintings that i thin...
1,sadness,"Yet the dog had grown old and less capable , a...","yet the dog had grown old and less capable , a..."
2,fear,When I get into the tube or the train without ...,when i get into the tube or the train without ...
3,fear,This last may be a source of considerable disq...,this last may be a source of considerable disq...
4,anger,She disliked the intimacy he showed towards so...,she disliked the intimacy he showed towards so...
...,...,...,...
7929,sadness,When I felt alone and without love.,when i felt alone and without love.
7930,anger,Irina hung up in exasperation .,irina hung up in exasperation .
7931,sadness,No wonder she was now inconsolable at the pros...,no wonder she was now inconsolable at the pros...
7932,fear,My friend had been telling me about a certain ...,my friend had been telling me about a certain ...


In [ ]:
# Lemmatize words
lemmatizer = WordNetLemmatizer()
data['clean_text'] = data['clean_text'].apply(
    lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x)
    )

In [39]:
import re
from nltk.tokenize import word_tokenize

In [40]:
def clean_text(data):
    # remove hashtags and @usernames
    data = re.sub(r"(#[\d\w\.]+)", '', data)
    data = re.sub(r"(@[\d\w\.]+)", '', data)
    # tekenization using nltk
    data = word_tokenize(data)
    return data

In [41]:
texts = [' '.join(clean_text(text)) for text in train_data.clean_text]

In [42]:
texts

['there are tons of other paintings that i think are better .',
 'yet the dog had grown old and less capable , and one day the gillie had come and explained with great sorrow that the dog had suffered a stroke , and must be put down .',
 'when i get into the tube or the train without paying for the ticket .',
 'this last may be a source of considerable disquiet and one might not at first see how such obviously ` immoral `` content could be defended as part of a system of morality .',
 'she disliked the intimacy he showed towards some of them , was resentful of the memories they shared of which she was not a part , and felt excluded .',
 "when my family heard that my mother 's cousin who lives in england wrote us to tell that he had cancer of the lymph glands .",
 "finding out i am chosen to collect norms for chinese aphasia ( i will contribute to china 's catching up with the west in neuropsychology ) .",
 'a spokesperson said : ` glen is furious that the new ` anarchy `` promo feature

In [45]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [47]:
texts = [' '.join(clean_text(text)) for text in train_data['clean_text']]

texts_train = [' '.join(clean_text(text)) for text in train_data['clean_text']]

In [48]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

sequence_train = tokenizer.texts_to_sequences(texts_train)

index_of_words = tokenizer.word_index

# vacab size is number of unique words + reserved 0 index for padding
vocab_size = len(index_of_words) + 1

print('Number of unique words: {}'.format(len(index_of_words)))

Number of unique words: 9894
